In [ ]:
pip install jiwer

In [ ]:
import numpy as np
import jiwer
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
import unicodedata
import csv

In [ ]:
df=pd.read_csv(r'C:\\Users\user\Downloads\TranscriptionFinal.csv', encoding='latin1')
df.head(45)

In [ ]:
ground_truth=df.content.values.tolist()
google_hypothesis=df.google_transcription.values.tolist()
microsoft_hypothesis=df.msft_transcription.values.tolist()
amazon_hypothesis=df.amazon_transcription.values.tolist()

ASR=[google_hypothesis, microsoft_hypothesis, amazon_hypothesis]
ASR_hyp=[]+google_hypothesis+microsoft_hypothesis+amazon_hypothesis

r=len(ground_truth)

In [ ]:
#pre_processing the data
for i in range(len(ASR)):  
    ASR[i]=jiwer.RemoveMultipleSpaces()(ASR[i])
    ASR[i]=jiwer.RemoveWhiteSpace(replace_by_space=True)(ASR[i])
    ASR[i]=jiwer.ToLowerCase()(ASR[i])
    ASR[i]=jiwer.RemovePunctuation()(ASR[i])
    for j in range(r):
        ASR[i][j] = ASR[i][j].replace('\xa0', ' ')


In [ ]:
def wer(ref, hyp ,debug=True):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
    
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
    
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
    
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
    #    print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return wer_result, numCor, numSub, numIns, numDel, len(r);

In [ ]:
for i in range(len(ASR)):
    print("ASR number",i)
    Wer=[]
    NumCor=[]
    NumSub=[]
    NumIns=[]
    NumDel=[]
    WCount=[]
    for j in range(r):
        wer_result, numCor, numSub, numIns, numDel, wCount=wer(ground_truth[j], ASR[i][j] ,debug=True)
        #print("for recording number",i," the word error rate is", wer(ground_truth[i], google_hypothesis[i] ,debug=True))
        Wer.append(wer_result)
        NumCor.append(numCor)
        NumSub.append(numSub)
        NumIns.append(numIns)
        NumDel.append(numDel)    
        WCount.append(wCount)
    print("this is the WER:",Wer)
    print("this is the number of corecct words:",NumCor)
    print("this is the number of subsitutions:",NumSub)
    print("this is the number of insertions:",NumIns)
    print("this is the number of deletions:",NumDel)
    print("this is the number of words:",WCount)

In [ ]:
def wers(refs, hyps):
    numSub = 0
    numDel = 0
    numCor = 0
    numIns = 0
    numCount = 0
    for ref, hyp in zip(refs, hyps):
        wer_result, numCor, numSub, numIns, numDel, wCount = wer(ref, hyp, False)
        numSub += numSub
        numDel += numDel
        numCor += numCor
        numIns += numIns
        numCount += wCount
 
    return round( (numSub + numDel + numIns) / (float) (numCount), 3)

In [ ]:
for i in range(len(ASR)):
    avg_wer=wers(ground_truth, ASR[i])
    print(avg_wer)

In [ ]:
avg_wer=wers(ground_truth,ASR_hyp)
print(avg_wer)